# Possible SPN Enumeration
Detects Service Principal Name Enumeration used for Kerberoasting

## Rule Content
```
- title: Possible SPN Enumeration
  id: 1eeed653-dbc8-4187-ad0c-eeebb20e6599
  description: Detects Service Principal Name Enumeration used for Kerberoasting
  status: experimental
  references:
  - https://p16.praetorian.com/blog/how-to-use-kerberoasting-t1208-for-privilege-escalation
  author: Markus Neis, keepwatch
  date: 2018/11/14
  tags:
  - attack.credential_access
  - attack.t1208
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection_image:
      Image: '*\setspn.exe'
    selection_desc:
      Description: '*Query or reset the computer* SPN attribute*'
    cmd:
      CommandLine: '*-q*'
    condition: (selection_image or selection_desc) and cmd
  falsepositives:
  - Administrator Activity
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_path.keyword:*\\setspn.exe OR file_description.keyword:*Query\ or\ reset\ the\ computer*\ SPN\ attribute*) AND process_command_line.keyword:*\-q*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()